# Tweets (Sältzer)

In [ ]:
import polars as pl
import pandas as pd
import os
from pathlib import Path

In [ ]:
# Convert tweets to parquet because of a bug in polars
# if not os.path.isfile("tweets.parquet"):
df = pd.read_feather("../../../data/tweets/tweets.feather")
#   df.to_parquet("tweets.parquet")
  
# df = pl.read_parquet("tweets.parquet")
df.head()

In [ ]:
df.describe()